In [2]:
import nltk
import pandas as pd
import numpy as np
import os, sys
PATH = os.path.abspath("")[:-3]

In [4]:
# SOLO HACER LA PRIMERA VEZ QUE SE TIRA
# Se va a abrir en una ventana aparte de python, le das a download todo y cuando acabe la cierras.
# Si todo ha salido guay devuelve True.
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

# Preparar datos para predecir un usuario

In [11]:

# SCRAPPEAR UN USUARIO
from scrap import scrap_user

USER = "ComicGirlAshley"
tweets = scrap_user(USER)
tweets.to_csv(PATH+"data/example.csv", index=False) # Se guardan los tweets en un csv pero no hace falta lol.

In [12]:
# CÁLCULO DEL FACTOR REP
from sklearn.feature_extraction.text import CountVectorizer
#tweets = pd.read_csv(PATH+"data/example.csv") # Se puede leer el csv de antes.
contents = tweets["content"]
toks = []
sw = nltk.corpus.stopwords
stemmer = nltk.stem.snowball.EnglishStemmer()
# Tokenizar, stemmear, filtrar stop-words.
for content in contents.values:
    tk = nltk.word_tokenize(content)
    filtered_sentence = [w for w in tk if not w.lower() in sw.words('english')]
    toks.append([stemmer.stem(word) for word in filtered_sentence])
# Se cuenta número de repeticiones de palabras y se suman.
try:
    vect = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False, ngram_range=(1, 15), min_df=0.1).fit_transform(toks)
    factor_rep = vect.sum()
except:
    factor_rep = 0
print(factor_rep)

340


c:\Users\maria\Documents\Uni\ia orgs\bot_detector\.venv\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [13]:
# REPETICIÓN DE LAS FECHAS
from datetime import datetime
date = tweets["date"]
hours = []
for h in date.values:
    hours.append(pd.to_datetime(h).hour)
valores_finales=[]
tot= len(hours)
result = dict()
#contando el numero de repeticiones
for j in hours:
    if j not in result:
        result[j] = 0 
    result[j] += 1

#viendo el mayor valor y computando porcentaje
values=[]
mayor=0
for dato, valor in result.items():
    if valor > mayor:
        mayor= valor
hora_rep = mayor/tot
print(hora_rep)

0.1568627450980392


In [14]:
# DATOS MEDIOS
avg_likes = np.average(tweets["likeCount"].values)
avg_retweets = np.average(tweets["retweetCount"].values)
avg_reply = np.average(tweets["replyCount"].values)
avg_quote = np.average(tweets["quoteCount"].values)

In [15]:
# El usuario listo para predecir se guarda en user_data.
user_data = tweets[["verified", "followersCount", "friendsCount", "tweetsCount", "listedCount", "mediaCount"]].iloc[-1, :].values.tolist()
user_data.insert(0, factor_rep)
user_data.extend([avg_reply, avg_retweets, avg_likes, avg_quote, hora_rep, user_data[2]-user_data[3]])
print(user_data)
print(len(user_data))

[340, False, 26626, 97, 19968, 134, 6255, 4.637254901960785, 17.470588235294116, 271.45098039215685, 0.8921568627450981, 0.1568627450980392, 26529]
13


# Modelo

In [8]:
# Se leen los datos generados con model.py
training = pd.read_csv(PATH+"data/training_new.csv").sample(frac=1).reset_index(drop=True)
x_train = training.iloc[:, 1:-1]
y_train = training.iloc[:, -1:]
print(x_train.head())
print(y_train.head())

   factor_rep  verified  followersCount  friendsCount  tweetsCount  \
0         204         0              34           356          457   
1         208         0          364588        359855        30978   
2         206         0            7719          2678       231028   
3         182         0              23            50         2968   
4         225         0              48           222          588   

   listedCount  mediaCount  avg_reply  avg_retweet  like_reply  avg_quote  \
0            1          18   0.187500     0.020833    0.583333   0.000000   
1         3997        6244   0.977273     1.204545   11.431818   0.068182   
2          431       28121   0.479167     0.291667    5.500000   0.062500   
3            1           0   0.600000     0.000000    0.200000   0.000000   
4            1          35   0.342857     0.028571    1.914286   0.000000   

   hora_rep  followers_diff  
0  0.270833             -34  
1  0.136364         -364588  
2  0.166667           -771

## Regresión logística

In [9]:
# Regresión logística, aquí habría que probar perceptrones y demás
from sklearn.linear_model import LogisticRegression
model = LogisticRegression().fit(x_train, np.array(y_train.values.T[0]))

In [16]:
# Imprime la probabilidad de que el usuario preparado antes sea bot.
print(model.predict_proba(np.array([user_data]))[0])
print(model.predict(np.array([user_data]))[0])

[0.47215778 0.52784222]
human


c:\Users\maria\Documents\Uni\ia orgs\bot_detector\.venv\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\maria\Documents\Uni\ia orgs\bot_detector\.venv\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
